<a href="https://colab.research.google.com/github/BrunoDkno368/ETL-de-Limpieza-y-Normalizaci-n-de-Datos-de-Clientes-con-Reglas-de-Negocio/blob/main/normalizac_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ETL

In [2684]:
import pandas as pd
import numpy as np
import re


Reglas de negocio por columna

🔹 id_cliente

Reglas

Debe ser único

No puede ser nulo

Tipo entero

Acciones ETL

Verificar duplicados

Mantener como int

🔹 nombre

Reglas

No puede contener símbolos especiales

No puede estar vacío

Debe tener formato texto limpio

Si no hay dato válido → "Sin datos"

Acciones ETL

Limpieza de caracteres no alfabéticos

Normalización de espacios

Corrección manual de errores comunes

Imputación controlada

🔹 edad

Reglas

Rango válido: 18 a 100 años

No se permiten textos

Valores fuera de rango → inválidos

Acciones ETL

Conversión a numérico

Reemplazo de valores inválidos por NaN

Imputación con media o mediana

Conversión final a entero

Regla típica bancaria: no se opera con menores de edad.

🔹 email

Reglas

Debe contener @ y dominio

Emails inválidos → NaN

Emails duplicados permitidos (mismo cliente)

Acciones ETL

Validación con regex

Normalización a minúsculas

Valores inválidos → NaN

🔹 saldo_cuenta

Reglas

No puede ser texto

No puede ser negativo

Máximo permitido: 10.000.000

Valores fuera de regla → NaN

Acciones ETL

Normalización de separadores decimales

Conversión a numérico

Detección de outliers

Imputación con media o mediana

🔹 tipo_cuenta

Reglas

Valores permitidos:

Caja Ahorro

Cuenta Corriente

Acciones ETL

Mapeo semántico:

CA, CajaAhorro → Caja Ahorro

CC, Cuenta_Corriente → Cuenta Corriente

Valores desconocidos → Sin datos

🔹 fecha_alta

Reglas

Formato final: YYYY-MM-DD

No puede ser futura

Fechas inválidas → NaT

Acciones ETL

Parsing múltiple

Traducción de meses

Conversión a datetime

🔹 estado_cliente

Reglas

Valores permitidos:

Activo

Inactivo

Suspendido

Valores vacíos → Inactivo

Acciones ETL

Normalización de texto

Validación categórica

##EXTRAT (E)

In [2685]:
df = pd.read_csv('/content/drive/MyDrive/Practica 2 etl /banco_dataset_sucio.csv')

In [2686]:
df

,id_cliente,nombre,edad,email,saldo_cuenta,tipo_cuenta,fecha_alta,estado_cliente
0,101,Juan Perez,35,juan@mail.com,150000.50,Caja Ahorro,2022-01-05,Activo
1,102,Mar ia Lopez,treinta,maria@,-5000,caja ahorro,05/01/2022,activo
2,103,Ju@n Gómez,-2,juan@gmail.com,30000,Cuenta Corriente,enero 5 2022,ACTIVO
3,104,NaN,45,NaN,"1.200.000,75",CA,2022-13-01,Inactivo
4,105,NaN,200,ana@mail.com,99999999,NaN,NaN,NaN
5,106,Ana Torres,NaN,ana@mail.com,NaN,Cuenta_Corriente,2022-01-32,INACTIVO
6,107,Carlos,29,carlos@mail,45000,CajaAhorro,2022/01/05,Activo
7,108,Lu!sa Méndez,40,luisa@mail.com,abc,CC,2022-01-05 10:30,Suspendido
8,109,Pedro,17,pedro@mail.com,0,Caja Ahorro,2022-1-5,Activo
9,110,Maria,abc,maria@mail.com,25000.30,Cuenta Corriente,febrero 30 2022,Inactivo


##TRANSFORM (T)

In [2687]:
df.columns

Index(['id_cliente', 'nombre', 'edad', 'email', 'saldo_cuenta', 'tipo_cuenta',
       'fecha_alta', 'estado_cliente'],
      dtype='object')

###COLUMNA ID CLIENTE


**Reglas de negocio**

1 Debe ser único

2 No puede ser nulo

3 Tipo entero


que no sea nulo

In [2688]:
df =df[df['id_cliente'].notna()]


Que sea numero entero

In [2689]:
df['id_cliente']= df['id_cliente'].astype(int)


que no tenga duplicados

In [2690]:
df['id_cliente'].drop_duplicates()

,id_cliente
0,101
1,102
2,103
3,104
4,105
5,106
6,107
7,108
8,109
9,110


confirmamos que cumplimos con las reglas de negocio

In [2691]:
df['id_cliente'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 12 entries, 0 to 11
Series name: id_cliente
Non-Null Count  Dtype
--------------  -----
12 non-null     int64
dtypes: int64(1)
memory usage: 228.0 bytes


### COLUMNA NOMBRE

**Reglas de negocio**

1 No puede contener símbolos especiales

2 No puede estar vacío

3 Debe tener formato texto limpio

4 Si no hay dato válido → "Sin datos"


In [2692]:
print(df['nombre'])

0       Juan Perez
1     Mar ia Lopez
2       Ju@n Gómez
3              NaN
4              NaN
5      Ana  Torres
6           Carlos
7     Lu!sa Méndez
8            Pedro
9            Maria
10           M@rio
11         Laura  
Name: nombre, dtype: object


verificamos varios problemas en los nombres desde simbolos hasta espacios vamos a comenzar por esta columna a normalizar

pasamos toda la columna a string StringDtype .astype('string') para poder manipular los datos nulos y los que tiene simbolo

In [2693]:
df['nombre_corregido'] = df['nombre'].astype('string')

In [2694]:
df['nombre_corregido'] = (
    df['nombre']
    .astype('string')
    # reemplazos semánticos
    .str.replace('@', 'a', regex=False)
    .str.replace('!', 'i', regex=False)
    # solo letras, espacios y acentos
    .str.replace(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑ\s]', '', regex=True)
    # normalizar espacios
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
    # 🔴 unir palabras cortadas: "Mar ia" → "Maria"
    .str.replace(r'\b([A-Za-zÁÉÍÓÚáéíóúñÑ]{2,})\s+([a-z]{1,2})\b', r'\1\2', regex=True)
    # capitalizar al final
    .str.title()
)

df['nombre_corregido'] = df['nombre_corregido'].fillna('Sin datos')


Eliminamos todos los simbolos

In [2695]:
df['nombre_corregido'] = df['nombre_corregido'].fillna('Sin datos')

In [2696]:
print(df[['nombre','nombre_corregido']])

          nombre nombre_corregido
0     Juan Perez       Juan Perez
1   Mar ia Lopez      Maria Lopez
2     Ju@n Gómez       Juan Gómez
3            NaN        Sin datos
4            NaN        Sin datos
5    Ana  Torres       Ana Torres
6         Carlos           Carlos
7   Lu!sa Méndez     Luisa Méndez
8          Pedro            Pedro
9          Maria            Maria
10         M@rio            Mario
11       Laura              Laura


asignamos la columna que trabajamos a la columna original

In [2697]:
df['nombre']= df['nombre_corregido']
df.drop('nombre_corregido', axis=1, inplace=True)
print(df['nombre'])

0       Juan Perez
1      Maria Lopez
2       Juan Gómez
3        Sin datos
4        Sin datos
5       Ana Torres
6           Carlos
7     Luisa Méndez
8            Pedro
9            Maria
10           Mario
11           Laura
Name: nombre, dtype: string


###COLUMNA EDAD

**Reglas de negocio**

Rango válido: 18 a 100 años

No se permiten textos

Valores fuera de rango → inválidos

Acciones ETL

Conversión a numérico

Reemplazo de valores inválidos por NaN

Imputación con media o mediana

Conversión final a entero

Regla típica bancaria: no se opera con menores de edad.

verificamos los errores que posee la columna edad, num negativos, en palabras, nan y fuera de parametros

In [2698]:
print(df['edad'])

0          35
1     treinta
2          -2
3          45
4         200
5         NaN
6          29
7          40
8          17
9         abc
10         60
11         33
Name: edad, dtype: object


cambiamos a string para poder manipular los datos

In [2699]:
df['edad_corregida'] = df['edad'].astype('string')

creamos un diccionario para cambia trinta a 30 por un tema de escalabilidad, control

In [2700]:
map_edad = {'treinta':'30'}

mapeamos en el df

In [2701]:
df['edad_corregida'] = df['edad_corregida'].replace(map_edad)

pasamos a numerico

ajustamos la condicion que la edad sea entre 18 y 100

In [2702]:
df['edad_corregida'] = pd.to_numeric(df['edad_corregida'], errors='coerce')

In [2703]:
df.loc[
    ~df['edad_corregida'].between(18, 100),
    'edad_corregida'
] = pd.NA

In [2704]:
df['edad_corregida'] = df['edad_corregida'].fillna(df['edad_corregida'].median())

In [2705]:
df[['edad', 'edad_corregida']]

,edad,edad_corregida
0,35,35
1,treinta,30
2,-2,35
3,45,45
4,200,35
5,NaN,35
6,29,29
7,40,40
8,17,35
9,abc,35


In [2706]:
df['edad']= df['edad_corregida']
df.drop('edad_corregida', axis=1, inplace=True)

###COLUMNA EMAIL

Reglas de negocio

Debe contener @ y dominio

Emails inválidos → NaN

Emails duplicados permitidos (mismo cliente)

Acciones ETL

Validación con regex

Normalización a minúsculas

Valores inválidos → NaN

In [2707]:
print(df['email'])

0      juan@mail.com
1             maria@
2     juan@gmail.com
3                NaN
4       ana@mail.com
5       ana@mail.com
6        carlos@mail
7     luisa@mail.com
8     pedro@mail.com
9     maria@mail.com
10               NaN
11    laura@mail.com
Name: email, dtype: object


como vemos email es tipo objero y lo pasamos a string para poder manipularlo mejor

In [2708]:
df['email_corregido'] = df['email'].astype('string')

normalizamos todo a minuscula

In [2709]:
df['email_corregido'] = df['email_corregido'].str.lower()

validamos mail con regex

condicion: ^[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}$

In [2710]:
mask_email_valido = df['email_corregido'].str.match(
    r'^[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}$',
    na=False
)

ejecutamos para la validacion y cargamos en el campo mail

In [2711]:
df.loc[~mask_email_valido, 'email_corregido'] = pd.NA

In [2712]:
df['email_corregido'] = df['email_corregido'].fillna('Sin datos')

In [2713]:
df[['email', 'email_corregido']]

,email,email_corregido
0,juan@mail.com,juan@mail.com
1,maria@,Sin datos
2,juan@gmail.com,juan@gmail.com
3,NaN,Sin datos
4,ana@mail.com,ana@mail.com
5,ana@mail.com,ana@mail.com
6,carlos@mail,Sin datos
7,luisa@mail.com,luisa@mail.com
8,pedro@mail.com,pedro@mail.com
9,maria@mail.com,maria@mail.com


In [2714]:
df['email'] = df['email_corregido']
df.drop('email_corregido', axis=1, inplace=True)

In [2715]:
df

,id_cliente,nombre,edad,email,saldo_cuenta,tipo_cuenta,fecha_alta,estado_cliente
0,101,Juan Perez,35,juan@mail.com,150000.50,Caja Ahorro,2022-01-05,Activo
1,102,Maria Lopez,30,Sin datos,-5000,caja ahorro,05/01/2022,activo
2,103,Juan Gómez,35,juan@gmail.com,30000,Cuenta Corriente,enero 5 2022,ACTIVO
3,104,Sin datos,45,Sin datos,"1.200.000,75",CA,2022-13-01,Inactivo
4,105,Sin datos,35,ana@mail.com,99999999,NaN,NaN,NaN
5,106,Ana Torres,35,ana@mail.com,NaN,Cuenta_Corriente,2022-01-32,INACTIVO
6,107,Carlos,29,Sin datos,45000,CajaAhorro,2022/01/05,Activo
7,108,Luisa Méndez,40,luisa@mail.com,abc,CC,2022-01-05 10:30,Suspendido
8,109,Pedro,35,pedro@mail.com,0,Caja Ahorro,2022-1-5,Activo
9,110,Maria,35,maria@mail.com,25000.30,Cuenta Corriente,febrero 30 2022,Inactivo


###COLUMNA SALDO EN CUENTA

🔹 saldo_cuenta

Reglas de negocio

No puede ser texto

No puede ser negativo

Máximo permitido: 10.000.000

Valores fuera de regla → NaN


In [2716]:
print(df['saldo_cuenta'])

0        150000.50
1            -5000
2            30000
3     1.200.000,75
4         99999999
5              NaN
6            45000
7              abc
8                0
9         25000.30
10         1000000
11                
Name: saldo_cuenta, dtype: object


unificamos los . y las , para que todo quede un mismo formato

In [2717]:
df['saldo_corregido'] = df['saldo_cuenta'].str.replace('.','', regex=False)

In [2718]:
df['saldo_corregido'] = df['saldo_corregido'].str.replace(',','.', regex=False)

pasamos toda la columna a numerico

In [2719]:
df['saldo_corregido'] = pd.to_numeric(df['saldo_corregido'], errors='coerce')

verificamos las condiciones para que sea entre 0 a 10.000.000

In [2720]:
df.loc[
    ~df['saldo_corregido'].between(0, 10_000_000),
    'saldo_corregido'
] = pd.NA

segun regla de negocio cambiar los Nan a mediana

In [2721]:
df['saldo_corregido']= df['saldo_corregido'].fillna(df['saldo_corregido'].median())

In [2722]:
df[['saldo_cuenta','saldo_corregido']]

,saldo_cuenta,saldo_corregido
0,150000.50,522500.00
1,-5000,522500.00
2,30000,30000.00
3,"1.200.000,75",1200000.75
4,99999999,522500.00
5,NaN,522500.00
6,45000,45000.00
7,abc,522500.00
8,0,0.00
9,25000.30,2500030.00


In [2723]:
df['saldo_cuenta']= df['saldo_corregido']
df.drop('saldo_corregido', axis=1, inplace=True)

In [2724]:
df

,id_cliente,nombre,edad,email,saldo_cuenta,tipo_cuenta,fecha_alta,estado_cliente
0,101,Juan Perez,35,juan@mail.com,522500.00,Caja Ahorro,2022-01-05,Activo
1,102,Maria Lopez,30,Sin datos,522500.00,caja ahorro,05/01/2022,activo
2,103,Juan Gómez,35,juan@gmail.com,30000.00,Cuenta Corriente,enero 5 2022,ACTIVO
3,104,Sin datos,45,Sin datos,1200000.75,CA,2022-13-01,Inactivo
4,105,Sin datos,35,ana@mail.com,522500.00,NaN,NaN,NaN
5,106,Ana Torres,35,ana@mail.com,522500.00,Cuenta_Corriente,2022-01-32,INACTIVO
6,107,Carlos,29,Sin datos,45000.00,CajaAhorro,2022/01/05,Activo
7,108,Luisa Méndez,40,luisa@mail.com,522500.00,CC,2022-01-05 10:30,Suspendido
8,109,Pedro,35,pedro@mail.com,0.00,Caja Ahorro,2022-1-5,Activo
9,110,Maria,35,maria@mail.com,2500030.00,Cuenta Corriente,febrero 30 2022,Inactivo


###COLUMNA TIPO DE CUENTA

In [2725]:
df['tipo_cuenta']

,tipo_cuenta
0,Caja Ahorro
1,caja ahorro
2,Cuenta Corriente
3,CA
4,NaN
5,Cuenta_Corriente
6,CajaAhorro
7,CC
8,Caja Ahorro
9,Cuenta Corriente


In [2726]:
df['tipo_cuenta'].dtypes

dtype('O')

In [2727]:
df['cuenta_corregida']= df['tipo_cuenta'].astype('string')

In [2728]:
df['cuenta_corregida']= df['cuenta_corregida'].str.lower()

In [2729]:
print(df['cuenta_corregida'].unique())

<StringArray>
[     'caja ahorro', 'cuenta corriente',               'ca',
               <NA>, 'cuenta_corriente',       'cajaahorro',
               'cc']
Length: 7, dtype: string


In [2730]:
cuentas= {'ca':'caja ahorro',
          'cajaahorro':'caja ahorro',
          'ca':'caja ahorro',
          'cuenta_corriente': 'cuenta corriente',
          'cc':'cuenta corriente'}

In [2731]:
df['cuenta_corregida']= df['cuenta_corregida'].replace(cuentas)

In [2732]:
df['cuenta_corregida']= df['cuenta_corregida'].fillna('Sin datos')

In [2733]:
df['cuenta_corregida']= df['cuenta_corregida'].str.title()

In [2734]:
df[['tipo_cuenta','cuenta_corregida']]

,tipo_cuenta,cuenta_corregida
0,Caja Ahorro,Caja Ahorro
1,caja ahorro,Caja Ahorro
2,Cuenta Corriente,Cuenta Corriente
3,CA,Caja Ahorro
4,NaN,Sin Datos
5,Cuenta_Corriente,Cuenta Corriente
6,CajaAhorro,Caja Ahorro
7,CC,Cuenta Corriente
8,Caja Ahorro,Caja Ahorro
9,Cuenta Corriente,Cuenta Corriente


In [2735]:
df['tipo_cuenta']= df['cuenta_corregida']
df.drop('cuenta_corregida',axis=1, inplace=True)

In [2736]:
df

,id_cliente,nombre,edad,email,saldo_cuenta,tipo_cuenta,fecha_alta,estado_cliente
0,101,Juan Perez,35,juan@mail.com,522500.00,Caja Ahorro,2022-01-05,Activo
1,102,Maria Lopez,30,Sin datos,522500.00,Caja Ahorro,05/01/2022,activo
2,103,Juan Gómez,35,juan@gmail.com,30000.00,Cuenta Corriente,enero 5 2022,ACTIVO
3,104,Sin datos,45,Sin datos,1200000.75,Caja Ahorro,2022-13-01,Inactivo
4,105,Sin datos,35,ana@mail.com,522500.00,Sin Datos,NaN,NaN
5,106,Ana Torres,35,ana@mail.com,522500.00,Cuenta Corriente,2022-01-32,INACTIVO
6,107,Carlos,29,Sin datos,45000.00,Caja Ahorro,2022/01/05,Activo
7,108,Luisa Méndez,40,luisa@mail.com,522500.00,Cuenta Corriente,2022-01-05 10:30,Suspendido
8,109,Pedro,35,pedro@mail.com,0.00,Caja Ahorro,2022-1-5,Activo
9,110,Maria,35,maria@mail.com,2500030.00,Cuenta Corriente,febrero 30 2022,Inactivo


###COLUMNA FECHA

🔹 fecha_alta

Reglas

Formato final: YYYY-MM-DD

No puede ser futura

Fechas inválidas → NaT


In [2737]:
print(df['fecha_alta'])

0           2022-01-05
1           05/01/2022
2         enero 5 2022
3           2022-13-01
4                  NaN
5           2022-01-32
6           2022/01/05
7     2022-01-05 10:30
8             2022-1-5
9      febrero 30 2022
10          2021-12-15
11          15-12-2021
Name: fecha_alta, dtype: object


In [2738]:
df['fecha_corregida'] = df['fecha_alta'].astype('string').str.lower().str.strip()

mapeamos los nombre en ingles

In [2739]:
meses = {
    'enero': '1',
    'febrero': '2',
    'marzo': '3',
    'abril': '4',
    'mayo': '5',
    'junio': '6',
    'julio': '7',
    'agosto': '8',
    'septiembre': '9',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

for es, en in meses.items():
    df['fecha_corregida'] = df['fecha_corregida'].str.replace(es, en, regex=False)


como tenemos fechas en diferentes formatos vamos a swapear para no perderla por ejemplo 2022-01-05 o 15-12-2021

In [2740]:
def parse_fecha(valor):
    if pd.isna(valor):
        return pd.NaT

    valor = str(valor).strip()

    # intento directo (ISO, con hora, con /)
    fecha = pd.to_datetime(valor, errors='coerce')
    if not pd.isna(fecha):
        return fecha.normalize()

    # intento dayfirst (05/01/2022, 15-12-2021)
    fecha = pd.to_datetime(valor, errors='coerce', dayfirst=True)
    if not pd.isna(fecha):
        return fecha.normalize()

    # intento formato con espacios: 1 5 2022
    fecha = pd.to_datetime(valor, errors='coerce', format='%d %m %Y')
    if not pd.isna(fecha):
        return fecha

    # corrección estructural YYYY-DD-MM → YYYY-MM-DD
    m = re.fullmatch(r'(\d{4})-(\d{2})-(\d{2})', valor)
    if m:
        y, a, b = m.groups()
        fecha = pd.to_datetime(f"{y}-{b}-{a}", errors='coerce')
        if not pd.isna(fecha):
            return fecha

    return pd.NaT

In [2741]:
df['fecha_corregida']= df['fecha_corregida'].apply(parse_fecha)

/tmp/ipython-input-2151615972.py:8: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  fecha = pd.to_datetime(valor, errors='coerce')


In [2742]:
df[['fecha_alta','fecha_corregida']]

,fecha_alta,fecha_corregida
0,2022-01-05,2022-01-05
1,05/01/2022,2022-05-01
2,enero 5 2022,2022-01-05
3,2022-13-01,2022-01-13
4,NaN,NaT
5,2022-01-32,NaT
6,2022/01/05,2022-01-05
7,2022-01-05 10:30,2022-01-05
8,2022-1-5,2022-01-05
9,febrero 30 2022,NaT


In [2743]:
df.fillna({'fecha_corregida':'Sin datos'}, inplace=True)

In [2744]:
df['fecha_alta']= df['fecha_corregida']
df.drop('fecha_corregida', axis=1, inplace=True)

In [2745]:
df

,id_cliente,nombre,edad,email,saldo_cuenta,tipo_cuenta,fecha_alta,estado_cliente
0,101,Juan Perez,35,juan@mail.com,522500.00,Caja Ahorro,2022-01-05 00:00:00,Activo
1,102,Maria Lopez,30,Sin datos,522500.00,Caja Ahorro,2022-05-01 00:00:00,activo
2,103,Juan Gómez,35,juan@gmail.com,30000.00,Cuenta Corriente,2022-01-05 00:00:00,ACTIVO
3,104,Sin datos,45,Sin datos,1200000.75,Caja Ahorro,2022-01-13 00:00:00,Inactivo
4,105,Sin datos,35,ana@mail.com,522500.00,Sin Datos,Sin datos,NaN
5,106,Ana Torres,35,ana@mail.com,522500.00,Cuenta Corriente,Sin datos,INACTIVO
6,107,Carlos,29,Sin datos,45000.00,Caja Ahorro,2022-01-05 00:00:00,Activo
7,108,Luisa Méndez,40,luisa@mail.com,522500.00,Cuenta Corriente,2022-01-05 00:00:00,Suspendido
8,109,Pedro,35,pedro@mail.com,0.00,Caja Ahorro,2022-01-05 00:00:00,Activo
9,110,Maria,35,maria@mail.com,2500030.00,Cuenta Corriente,Sin datos,Inactivo


###COLUMNA ESTADO CLIENTE

Reglas de negocio

Valores permitidos:

Activo

Inactivo

Suspendido

Valores vacíos → Inactivo


In [2746]:
print(df['estado_cliente'])

0         Activo
1         activo
2         ACTIVO
3       Inactivo
4            NaN
5       INACTIVO
6         Activo
7     Suspendido
8         Activo
9       Inactivo
10        Activo
11           NaN
Name: estado_cliente, dtype: object


In [2747]:
df['estado_corregido']= df['estado_cliente'].astype('string').str.lower().str.strip()

In [2748]:
df.fillna({'estado_corregido':'sin datos'}, inplace=True)

In [2749]:
df['estado_corregido'] = df['estado_corregido'].str.title()

In [2750]:
df['estado_cliente'] = df['estado_corregido']
df.drop('estado_corregido', axis=1, inplace=True)

In [2751]:
df

,id_cliente,nombre,edad,email,saldo_cuenta,tipo_cuenta,fecha_alta,estado_cliente
0,101,Juan Perez,35,juan@mail.com,522500.00,Caja Ahorro,2022-01-05 00:00:00,Activo
1,102,Maria Lopez,30,Sin datos,522500.00,Caja Ahorro,2022-05-01 00:00:00,Activo
2,103,Juan Gómez,35,juan@gmail.com,30000.00,Cuenta Corriente,2022-01-05 00:00:00,Activo
3,104,Sin datos,45,Sin datos,1200000.75,Caja Ahorro,2022-01-13 00:00:00,Inactivo
4,105,Sin datos,35,ana@mail.com,522500.00,Sin Datos,Sin datos,Sin Datos
5,106,Ana Torres,35,ana@mail.com,522500.00,Cuenta Corriente,Sin datos,Inactivo
6,107,Carlos,29,Sin datos,45000.00,Caja Ahorro,2022-01-05 00:00:00,Activo
7,108,Luisa Méndez,40,luisa@mail.com,522500.00,Cuenta Corriente,2022-01-05 00:00:00,Suspendido
8,109,Pedro,35,pedro@mail.com,0.00,Caja Ahorro,2022-01-05 00:00:00,Activo
9,110,Maria,35,maria@mail.com,2500030.00,Cuenta Corriente,Sin datos,Inactivo


##LOAD (L)

In [2752]:
df.to_csv('datos_limpios.csv', index=False)